# California housing dataset with Functional API
---

Try a regression problem, using the functional API which is more flexible

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
tf.__version__
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load the data
---

Note pytorch also has an API for loading and downloading datasets, but keras version used below is more explicit

In [10]:
housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit(x_train).transform(x_train)
x_valid, x_test = scaler.transform(x_valid), scaler.transform(x_test)

## Deep and wide model
---
A deep and wide model connects some or all of the inputs directly to output layer, as well as pass through deeper layers. This kind of model cannot be specified using just the sequential API

The semantics is similar, but all layers can be thought of as functions.

In [7]:
# define how the data flows through the model
input_ = keras.layers.Input(shape=x_train.shape[1:])
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_)
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(units=1)(concat)

# wrap in keras.Model
model = keras.Model(inputs=[input_], outputs=[output])

# compile as usual
model.compile(loss=keras.losses.MSE, optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]                

In [8]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_valid, y_valid))

Epoch 1/10
182/182 [==============================] - 5s 26ms/step - loss: 2.0755 - mae: 1.0201 - val_loss: 1.0108 - val_mae: 0.7417
Epoch 2/10
182/182 [==============================] - 5s 26ms/step - loss: 0.8191 - mae: 0.6581 - val_loss: 0.7539 - val_mae: 0.6319
Epoch 3/10
182/182 [==============================] - 4s 22ms/step - loss: 0.7048 - mae: 0.6080 - val_loss: 0.7068 - val_mae: 0.6065
Epoch 4/10
182/182 [==============================] - 3s 18ms/step - loss: 0.6677 - mae: 0.5885 - val_loss: 0.6765 - val_mae: 0.5905
Epoch 5/10
182/182 [==============================] - 3s 17ms/step - loss: 0.6395 - mae: 0.5733 - val_loss: 0.6511 - val_mae: 0.5807
Epoch 6/10
182/182 [==============================] - 4s 22ms/step - loss: 0.6164 - mae: 0.5640 - val_loss: 0.6322 - val_mae: 0.5688
Epoch 7/10
182/182 [==============================] - 3s 17ms/step - loss: 0.5966 - mae: 0.5531 - val_loss: 0.6131 - val_mae: 0.5617
Epoch 8/10
182/182 [==============================] - 3s 17ms/step - 

## Same model but specify which inputs get sent directly to ouput layer, and whic inputs pass through the model
---

In [12]:
x_train[0].shape

(8,)

In [12]:
# define the two input sizes
input_dim_1 = 5
input_dim_2 = 6

# define how the data flows through the model
input_1 = keras.layers.Input(shape=input_dim_1, name="wide_input")
input_2 = keras.layers.Input(shape=input_dim_2, name="deep_input")
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_2)
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_1, hidden2])
output = keras.layers.Dense(units=1)(concat)

# wrap in keras.Model
model = keras.Model(inputs=[input_1, input_2], outputs=[output])

# compile as usual
model.compile(loss=keras.losses.MSE, optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
______________________________________________________________________________________________

In [13]:
# set up the data splits
x_train_1, x_valid_1 = x_train[:,:input_dim_1], x_valid[:,:input_dim_1]
x_train_2, x_valid_2 = x_train[:,-input_dim_2:], x_valid[:,-input_dim_2:]

# train the model
history = model.fit(x=(x_train_1, x_train_2), y=y_train, batch_size=64, epochs=10, validation_data=((x_valid_1, x_valid_2), y_valid))

Epoch 1/10
182/182 [==============================] - 7s 36ms/step - loss: 3.2643 - mae: 1.4133 - val_loss: 1.4259 - val_mae: 0.8663
Epoch 2/10
182/182 [==============================] - 6s 33ms/step - loss: 1.1049 - mae: 0.7701 - val_loss: 0.8914 - val_mae: 0.7038
Epoch 3/10
182/182 [==============================] - 5s 26ms/step - loss: 0.8614 - mae: 0.6951 - val_loss: 0.7772 - val_mae: 0.6623
Epoch 4/10
182/182 [==============================] - 5s 27ms/step - loss: 0.7757 - mae: 0.6620 - val_loss: 0.7217 - val_mae: 0.6367
Epoch 5/10
182/182 [==============================] - 6s 31ms/step - loss: 0.7275 - mae: 0.6395 - val_loss: 0.6885 - val_mae: 0.6191
Epoch 6/10
182/182 [==============================] - 6s 32ms/step - loss: 0.6961 - mae: 0.6238 - val_loss: 0.6667 - val_mae: 0.6069
Epoch 7/10
182/182 [==============================] - 5s 26ms/step - loss: 0.6736 - mae: 0.6125 - val_loss: 0.6504 - val_mae: 0.5969
Epoch 8/10
182/182 [==============================] - 5s 28ms/step - 

## Subclassing API to build dynamic behaviour nets
---
Key points:

1. Subclass on `keras.Model`
2. Define any layers needed inside the constructor
3. Define how inputs flow through the model (including any dynamic behaviour) inside the call method

*Example* The below specifies a network similare to above, except that data is passed through the second hidden layer a random number of times between 1 and 3


In [24]:
class RandomNet(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.aux_output = keras.layers.Dense(1)
        self.main_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs  # inputs should be provides as tuple, just unpack it here
        input_deep = self.hidden1(input_deep)
        for _ in range(np.random.randint(3)):
            input_deep = self.hidden2(input_deep)
        concat = keras.layers.concatenate([input_wide, input_deep])
        output_aux = self.aux_output(input_deep)
        output_main = self.main_output(concat)
        return output_main, output_aux

model = RandomNet()
model.compile(loss=[keras.losses.MSE, keras.losses.MSE], optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['mae'])


In [25]:
model.fit(x=(x_train_1, x_train_2), y=(y_train, y_train), batch_size=64, epochs=10, validation_data=((x_valid_1, x_valid_2), y_valid, y_valid))


Epoch 1/10
182/182 [==============================] - 9s 47ms/step - loss: 6.5541 - output_1_loss: 3.1122 - output_2_loss: 3.4419 - output_1_mae: 1.3579 - output_2_mae: 1.4607 - val_loss: 11.9420 - val_output_1_loss: 5.4344 - val_output_2_loss: 6.5076 - val_output_1_mae: 0.8490 - val_output_2_mae: 0.9474
Epoch 2/10
182/182 [==============================] - 7s 39ms/step - loss: 2.4909 - output_1_loss: 1.1108 - output_2_loss: 1.3801 - output_1_mae: 0.7421 - output_2_mae: 0.8648 - val_loss: 6.8714 - val_output_1_loss: 2.9154 - val_output_2_loss: 3.9560 - val_output_1_mae: 0.6661 - val_output_2_mae: 0.8215
Epoch 3/10
182/182 [==============================] - 8s 45ms/step - loss: 1.8915 - output_1_loss: 0.7733 - output_2_loss: 1.1182 - output_1_mae: 0.6522 - output_2_mae: 0.8126 - val_loss: 5.8154 - val_output_1_loss: 2.3608 - val_output_2_loss: 3.4546 - val_output_1_mae: 0.6238 - val_output_2_mae: 0.7971
Epoch 4/10
182/182 [==============================] - 7s 38ms/step - loss: 1.7173 - 